<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Paul%20Himyak/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gdown

In [2]:
dataset_url = "https://drive.google.com/uc?export=download&id=1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg"
#dataset_url = "https://docs.google.com/spreadsheets/d/1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg/edit?usp=share_link&ouid=113369956649761615778&rtpof=true&sd=true"
dataset_filename = "profpatolog_modified.xlsx"
#!wget -qc dataset_url -O dataset_filename
gdown.download(dataset_url, output=dataset_filename, quiet=True )


'profpatolog_modified.xlsx'

In [276]:
df = pd.read_excel("profpatolog_modified.xlsx")


In [277]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [278]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

In [258]:
from datetime import datetime
class MyUtils:
  @staticmethod  
  # Определяем функцию для расчета возраста на основе даты
  def resolve_birthdate_to_age(date):
    current_year = pd.Timestamp.now().year
    age = current_year - date.year
    return age
  def resolve_genger(x):
    if x == 'Мужской':
        return FIELD_CLIENT_GENDER_M
    elif x == 'Женский':
        return FIELD_CLIENT_GENDER_F
    else:
        return None
  def resolve_conclusion(arg):
    try:
      if "Годен" == arg:
        return FIELD_CONCLUSION_FIT
      elif "Негоден" in arg:
        return FIELD_CONCLUSION_NOT_FIT
      elif "Годен" in arg:
        return FIELD_CONCLUSION_PART_FIT
      else:
        return FIELD_CONCLUSION_NOT_FIT
    except (TypeError):
        return None
  def resolve_dispansary_group(arg):
    try:
      if arg == "I":
        return FIELD_DISPANSARY_GROUP_I
      elif arg == "II":
        return FIELD_DISPANSARY_GROUP_II
      elif "III" in arg :
        return FIELD_DISPANSARY_GROUP_III
      else:
        return FIELD_DISPANSARY_GROUP_OTHER
    except (TypeError):
        return None

In [281]:
df[FIELD_CLIENT_AGE] = df[FIELD_CLIENT_BIRTH_DAY].apply(MyUtils.resolve_birthdate_to_age)

In [282]:
df[FIELD_CLIENT_GENDER] = df[FIELD_CLIENT_GENDER].apply(lambda x: MyUtils.resolve_genger(x) if pd.notnull(x) else x)

In [283]:
del df[FIELD_CLIENT_BIRTH_DAY]

In [284]:
df[FIELD_DISPANSARY_GROUP] = df[FIELD_DISPANSARY_GROUP].apply(MyUtils.resolve_dispansary_group)

In [285]:
df[FIELD_CONCLUSION] = df[FIELD_CONCLUSION].apply(lambda x: MyUtils.resolve_conclusion(x))

In [286]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns


In [287]:
df[mkb_columns] = df[mkb_columns].fillna(value='Z00.0')

In [288]:
# объединяем значения из нескольких столбцов в один массив, удаляем дубликаты
#df['mkb'] = df.apply(lambda row: list(set([row[mkb_col_name] for mkb_col_name in mkb_columns if not pd.isna(row[mkb_col_name]) ])), axis=1)
df['mkb'] = df.apply(lambda row: list(set([row[mkb_col_name] for mkb_col_name in mkb_columns])), axis=1)


In [289]:
df = df.loc[:, ~df.columns.str.contains('мкб10', case=False)]


In [279]:
df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].str.split(",")

In [290]:
df

,Unnamed: 0,КлиентПол,ЗаключениеМК,ДиспансернаяГруппа,ВредныеФакторы,КлиентВозраст,mkb
0,0,1,0.0,0.0,[11.4],47,[Z00.0]
1,1,0,0.0,2.0,[4.2.5],36,"[Z00.0, I11.9]"
2,2,0,1.0,1.0,[4.2.5],38,"[Z00.0, H52.1]"
3,3,0,0.0,0.0,[11.1],35,[Z00.0]
4,4,0,1.0,1.0,[11.1],39,"[H52.2, Z00.0]"
...,...,...,...,...,...,...,...
1977,1977,0,NaN,NaN,[6.1],51,[Z00.0]
1978,1978,0,NaN,NaN,[6.1],37,[Z00.0]
1979,1979,0,NaN,NaN,"[5.1, 23]",29,"[K02.1, Z00.0]"
1980,1980,1,NaN,NaN,[23],51,"[Z00.0, I11.9]"


In [102]:
df['mkb'].explode().unique()

array(['Z00.0', 'I11.9', 'H52.1', 'H52.2', 'H53.0', 'K29.3', 'I70',
       'H90.5', 'G90.9', 'H52.0', 'S68.2', 'N60.2', 'E11.9', 'K29.6',
       'M42.1', 'D25', 'I69.4', 'N83.2', 'H90.3', 'K44.9', 'N80.0',
       'G99.0', 'G12.1', 'K81.9', 'H11.0', 'G25.0', 'K26.9', 'K86.1',
       'H91.1', 'H74.1', 'I67.2', 'N28.1', 'I83.9', 'L40.0', 'T90.4',
       'H93.0', 'D25.9', 'I25.8', 'K80.5', 'F06.6', 'K40.9', 'E11.8',
       'E10.9', 'T94.0', 'F10.7', 'D35.2', 'L20.8', 'K29.5', 'H50.9',
       'K80.2', 'I25.2', 'E14.8', 'I67.9', 'Q12.0', 'S51.9', 'K02.1',
       'K00.0', 'F06.7', 'K03.66', 'K02.0', 'H25.0', 'I66.9', 'K29.8',
       'T92.0', 'F98.5', 'H40.1', 'H90.4', 'F41.0', 'R25.1', 'K25.7',
       'K29.7', 'J45.8', 'H47.2', 'J33.0', 'H66.1', 'K76.0', 'K04.5',
       'K80.1', 'H25.1', 'K25.9', 'K05.4', 'K26.7', 'H02.4', 'I49.9',
       'H01.0', 'I49.8', 'H35.0', 'H17.1', 'K43.6', 'J30.1', 'F07.0',
       'G58.9', 'J37.0', 'I25.9', 'H40.9', 'F70.0', 'G90.8', 'K08.2',
       'K82.8', 'K82.9'

In [291]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.array(df['mkb']))

In [292]:
xMkbIndexed = tokenizer.texts_to_sequences(df['mkb'])

In [293]:
mkb_max_len = max(len(sublist) for sublist in xMkbIndexed)
#mkb_max_len = max(xMkbIndexed, key=len)
print(mkb_max_len)

6


In [294]:
xMkbIndexedPadded = pad_sequences(xMkbIndexed, maxlen=mkb_max_len, padding='post')

In [295]:
xMkbIndexedPadded.shape

(1978, 6)

In [34]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: x.replace(".", "x"))

In [29]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].str.split(",")
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: ",".join(x))

In [296]:
tokenizer = Tokenizer()

In [297]:
tokenizer.fit_on_texts(df[FIELD_HARMFUL_FACTORS])

In [298]:
xTrainHarmfulIndexes = tokenizer.texts_to_sequences(df[FIELD_HARMFUL_FACTORS])

In [185]:
harm_max_len = max(len(sublist) for sublist in xTrainHarmfulIndexes)


In [299]:
xTrainHarmfulIndexesPadded = pad_sequences(xTrainHarmfulIndexes, maxlen=harm_max_len, padding='post')

In [300]:
xTrainHarmfulIndexesPadded.shape

(1978, 13)